# 공포/탐욕 지수 전문가 에이전트

사용자 입력 일자를 기준으로 공포/탐욕 지수 CSV 데이터를 참고해, 해당 기간의 매수/매도 판단에 대한 구조화된 의견을 제시합니다.

- **입력**: 평가하고 싶은 일자 (예: 2018-03-15)
- **출력**: Pydantic 스키마 (긍정적/보통/부정적, 시장 흐름 분석, 단기·장기 관점, 과거 주목 기간)

## 1. 환경 설정

In [124]:
from dotenv import load_dotenv

load_dotenv()

python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 9
python-dotenv could not parse statement starting at line 10
python-dotenv could not parse statement starting at line 11
python-dotenv could not parse statement starting at line 12
python-dotenv could not parse statement starting at line 13
python-dotenv could not parse statement starting at line 14
python-dotenv could not parse statement starting at line 15
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 17
python-dotenv could not parse statement starting at line 18
python-dotenv could not parse statement starting at line 19


True

In [125]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("fear-greed-expert-agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
fear-greed-expert-agent


In [126]:
import pandas as pd
from pathlib import Path
from typing import Literal, List, Dict

from langchain_core.prompts import load_prompt
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

## 2. 데이터 로드

CSV 경로는 상수로 두어 추후 API로 교체 시 `load_fear_greed_data()`만 수정하면 됩니다.

In [127]:
# 추후 API 교체 시 load_fear_greed_data() 내부만 수정
def get_csv_path():
    """노트북과 같은 디렉터리의 CSV 경로 (추후 API로 교체 가능)."""
    return Path("fear_greed_index.csv")

def load_fear_greed_data(csv_path: Path | None = None) -> pd.DataFrame:
    csv_path = csv_path or get_csv_path()
    df = pd.read_csv(csv_path)
    df["date"] = pd.to_datetime(df["date"])
    return df.sort_values("date").reset_index(drop=True)

df = load_fear_greed_data()
df.head(10)

,date,fear_greed_index
0,2018-02-19,67
1,2018-02-20,74
2,2018-02-21,54
3,2018-02-22,44
4,2018-02-23,39
5,2018-02-24,31
6,2018-02-25,33
7,2018-02-26,37
8,2018-02-27,44
9,2018-02-28,41


In [128]:
# 기간 추출: 선택된 날짜 이전 6개월 ~ 선택된 날짜 (추후 이 기간 데이터는 API로 받을 예정)
def get_period_data(df: pd.DataFrame, target_date: str, months_before: int = 6) -> str:
    target = pd.to_datetime(target_date)
    start = target - pd.DateOffset(months=months_before)
    end = target
    mask = (df["date"] >= start) & (df["date"] <= end)
    period = df.loc[mask, ["date", "fear_greed_index"]]
    lines = period.apply(lambda r: f"{r['date'].strftime('%Y-%m-%d')}: {r['fear_greed_index']}", axis=1)
    return "\n".join(lines.tolist())

# 예시 (선택일 이전 6개월 ~ 선택일)
get_period_data(df, "2018-03-15")

'2018-02-19: 67\n2018-02-20: 74\n2018-02-21: 54\n2018-02-22: 44\n2018-02-23: 39\n2018-02-24: 31\n2018-02-25: 33\n2018-02-26: 37\n2018-02-27: 44\n2018-02-28: 41\n2018-03-01: 38\n2018-03-02: 47\n2018-03-03: 56\n2018-03-04: 44\n2018-03-05: 55\n2018-03-06: 59\n2018-03-07: 37\n2018-03-08: 39\n2018-03-09: 37\n2018-03-10: 39\n2018-03-11: 40\n2018-03-12: 41\n2018-03-13: 41\n2018-03-14: 40\n2018-03-15: 32'

## 3. Pydantic 응답 스키마 및 Parser

In [129]:
# 과거 주목 기간 한 항목 (선택되는 기간은 최대 7일로 제한)
class NotablePeriod(BaseModel):
    title: str = Field(description="기간의 제목 (예: 공포에서 탐욕으로의 급격한 전환)")
    period_text: str = Field(
        description="기간 문자열. 반드시 최대 7일 이내 (예: 2019-02-17 ~ 2019-02-19)"
    )
    period_data: List[Dict[str, int]] = Field(
        description='각 날짜별 지수. 최대 7개 항목(최대 7일). 각 항목은 하나의 날짜(키)와 지수(값) 객체. 예: [{"2019-02-17": 38}, {"2019-02-18": 63}, {"2019-02-19": 65}]',
        max_length=7,
    )

# 전문가 에이전트 최종 응답
class FearGreedExpertResponse(BaseModel):
    verdict: Literal["긍정적", "보통", "부정적"] = Field(
        description="해당 일자 매수/매도 판단에 대한 종합 의견 (반드시 세 값 중 하나)"
    )
    market_flow_analysis: str = Field(
        description="해당 기간 시장 흐름 분석: 공포/탐욕 지수 추이와 국면 설명"
    )
    short_long_term_perspective: str = Field(
        description="단기적 관점과 장기적 관점을 각각 서술 (예: 단기: 해당 일자 전후 심리·추이, 장기: 6개월 구간에서의 위치·추세)"
    )
    notable_periods: List[NotablePeriod] = Field(
        description="과거 데이터에서 주목할 만한 기간과 그 수치 (항목 최대 5개, 각 기간은 최대 7일)"
    )

parser = PydanticOutputParser(pydantic_object=FearGreedExpertResponse)

## 4. 프롬프트, LLM, 체인

In [130]:
prompt = load_prompt("prompts/fear_greed_expert.yaml", encoding="utf-8")
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [131]:
# 전문가 품질 우선: gpt-4.1 (비용 절감 시 gpt-4.1-mini 로 변경)
MODEL_NAME = "gpt-4.1"
llm = ChatOpenAI(temperature=0, model=MODEL_NAME)
chain = prompt | llm | parser

## 5. 사용자 일자 입력 및 실행

In [132]:
# 사용자 입력: 평가하고 싶은 일자 (이전 6개월 ~ 선택일 데이터 전달, 추후 API에서 해당 기간 데이터 수신 예정)
target_date = "2019-02-19"
period_data = get_period_data(df, target_date, months_before=6)

In [133]:
response = chain.invoke({
    "target_date": target_date,
    "period_data": period_data,
})
response

FearGreedExpertResponse(verdict='부정적', market_flow_analysis='2019-02-19의 공포/탐욕 지수는 65로, 최근 며칠 사이 급격한 상승세를 보이며 극단적 탐욕 구간에 진입했습니다. 2월 17일 38에서 2월 18일 63, 2월 19일 65로 단기간에 27포인트 이상 급등하였으며, 이는 투자자 심리가 과열되어 있음을 시사합니다. 과거 6개월간 지수 흐름을 보면 60대 중반까지 오른 적이 거의 없으며, 이처럼 단기간 급등은 단기 조정 가능성을 높입니다.', short_long_term_perspective='단기: 최근 3일간 지수가 38→63→65로 급등해 단기적으로 과열 신호가 매우 강합니다. 이 구간에서는 추가 상승보다는 차익 실현 매도세가 유입될 가능성이 높으므로 매수에는 부정적입니다. 장기: 6개월 구간에서 65는 최상단에 해당하며, 과거에도 50대 중후반 이상에서는 이후 조정이 자주 발생했습니다. 장기적으로도 추가 상승 여력보다는 단기 과열 해소 구간 진입 가능성이 높아 보입니다.', notable_periods=[NotablePeriod(title='극단적 탐욕으로의 급격한 전환', period_text='2019-02-17 ~ 2019-02-19', period_data=[{'2019-02-17': 38}, {'2019-02-18': 63}, {'2019-02-19': 65}]), NotablePeriod(title='2019년 1월 초 단기 급등', period_text='2019-01-02 ~ 2019-01-04', period_data=[{'2019-01-02': 30}, {'2019-01-03': 33}, {'2019-01-04': 48}]), NotablePeriod(title='2018년 11월 중순~말 극단적 공포 구간', period_text='2018-11-21 ~ 2018-11-27', period_data=[{'2018-11-21': 15}, {'2018-11-22': 14}, {

In [134]:
# 필드별 출력
print("=== 종합 의견 (verdict) ===")
print(response.verdict)
print("\n=== 시장 흐름 분석 ===")
print(response.market_flow_analysis)
print("\n=== 단기·장기 관점 ===")
print(response.short_long_term_perspective)
print("\n=== 과거 주목 기간 ===")
for p in response.notable_periods:
    print(f"  - title: {p.title}")
    print(f"    period_text: {p.period_text}")
    print(f"    period_data: {p.period_data}")

=== 종합 의견 (verdict) ===
부정적

=== 시장 흐름 분석 ===
2019-02-19의 공포/탐욕 지수는 65로, 최근 며칠 사이 급격한 상승세를 보이며 극단적 탐욕 구간에 진입했습니다. 2월 17일 38에서 2월 18일 63, 2월 19일 65로 단기간에 27포인트 이상 급등하였으며, 이는 투자자 심리가 과열되어 있음을 시사합니다. 과거 6개월간 지수 흐름을 보면 60대 중반까지 오른 적이 거의 없으며, 이처럼 단기간 급등은 단기 조정 가능성을 높입니다.

=== 단기·장기 관점 ===
단기: 최근 3일간 지수가 38→63→65로 급등해 단기적으로 과열 신호가 매우 강합니다. 이 구간에서는 추가 상승보다는 차익 실현 매도세가 유입될 가능성이 높으므로 매수에는 부정적입니다. 장기: 6개월 구간에서 65는 최상단에 해당하며, 과거에도 50대 중후반 이상에서는 이후 조정이 자주 발생했습니다. 장기적으로도 추가 상승 여력보다는 단기 과열 해소 구간 진입 가능성이 높아 보입니다.

=== 과거 주목 기간 ===
  - title: 극단적 탐욕으로의 급격한 전환
    period_text: 2019-02-17 ~ 2019-02-19
    period_data: [{'2019-02-17': 38}, {'2019-02-18': 63}, {'2019-02-19': 65}]
  - title: 2019년 1월 초 단기 급등
    period_text: 2019-01-02 ~ 2019-01-04
    period_data: [{'2019-01-02': 30}, {'2019-01-03': 33}, {'2019-01-04': 48}]
  - title: 2018년 11월 중순~말 극단적 공포 구간
    period_text: 2018-11-21 ~ 2018-11-27
    period_data: [{'2018-11-21': 15}, {'2018-11-22': 14}, {'2018-11-23': 12}, {'2018-11-24': 15}, {'2018-11